In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from python_graphql_client import GraphqlClient

In [21]:
# Instantiate the client with an endpoint.
client = GraphqlClient(endpoint="https://api.thegraph.com/subgraphs/name/nounsdao/nouns-subgraph")

In [22]:
# Create the query string and variables required for the request.
query = """
    {
  nouns(first: 1000, orderBy: id) {
    id,
    owner,
    votes {
        proposal {
            id,
            proposer{id},
            proposalThreshold,
            quorumVotes,
            forVotes,
            againstVotes,
            abstainVotes,
            description,
            status,
            votes{id,support,votes,reason}

        },
        supportDetailed
    }
  }
}
"""
# Synchronous request
data = client.execute(query=query)

# For noun in data["data"]["nouns"] get each ['votes'] ['proposal'] and add them all to a list
proposals = []
for noun in data["data"]["nouns"]:
    for vote in noun["votes"]:
        proposals.append(vote["proposal"])


In [23]:
proposals_df = pd.DataFrame(proposals)
# Replace all "\n" with " " in the description column
proposals_df["description"] = proposals_df["description"].str.replace("\n", " ")
# each row of ['proposer'] is a dictionary {'id':'...'}, so we need to just keep the 'id'
proposals_df["proposer"] = proposals_df["proposer"].apply(lambda x: x["id"])


In [24]:
# change default row and column display limit to none
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [25]:
proposals_df = proposals_df.loc[proposals_df.astype(str).drop_duplicates().index]
proposals_df.shape


(93, 10)

In [26]:
proposals_df.head()

,id,proposer,proposalThreshold,quorumVotes,forVotes,againstVotes,abstainVotes,description,status,votes
0,1,0x2573c60a6d127755aa2dc85e342f7da2378a0cc5,0,1,7,0,0,# Donate 5 ETH each to 6 Charities *Past Disc...,EXECUTED,[{'id': '0x008c84421da5527f462886cec43d2717b68...
1,10,0xf476cd75be8fdd197ae0b466a2ec2ae44da41897,0,6,21,0,0,# Commission Jonathan Mann (@songadayman) to m...,EXECUTED,[{'id': '0x008c84421da5527f462886cec43d2717b68...
2,11,0x6f9e3976fa3b5b22761fe5d635e1f0d9d9aeb85d,0,6,20,0,0,# Nouns Comic # **Nouns Comic** **Nouns Com...,EXECUTED,[{'id': '0x008c84421da5527f462886cec43d2717b68...
3,12,0xae7f458667f1b30746354abc3157907d9f6fd15e,0,7,20,0,0,**Perpetual Glasses to Kids In Need V2** === *...,EXECUTED,[{'id': '0x2573c60a6d127755aa2dc85e342f7da2378...
4,14,0xf476cd75be8fdd197ae0b466a2ec2ae44da41897,0,8,25,0,0,# Brave Sponsored Takeover during NFT NYC ###...,EXECUTED,[{'id': '0x008c84421da5527f462886cec43d2717b68...


In [27]:
# Create a new dataframe then loop through all rows and for each row loop through all 'votes'
# Each vote that has a 'reason' we will append a new row to the new dataframe
# The new dataframe will have the columns 'proposal_id','quorumVotes','forVotes','againstVotes', 'abstainVotes','description','status','voter id','voter support','votes','reason'

reasons_df = pd.DataFrame()
for index, row in proposals_df.iterrows():
    for vote in row["votes"]:
        if vote["reason"] is not None:
            if vote['support']:
                # append using concat to add a new row to the dataframe into the correct columns saying "I support this proposal"
                reasons_df = pd.concat([reasons_df, pd.DataFrame([[row["id"], row["quorumVotes"], row["forVotes"], row["againstVotes"], row["abstainVotes"],"# Proposal: " + row["description"], row["status"], vote["id"], vote["support"], vote["votes"],"## I support this proposal: " + vote["reason"]]])], ignore_index=True)
            else:
                # append using concat to add a new row to the dataframe into the correct columns saying "I do not support this proposal"
                reasons_df = pd.concat([reasons_df, pd.DataFrame([[row["id"], row["quorumVotes"], row["forVotes"], row["againstVotes"], row["abstainVotes"],"# Proposal: " + row["description"], row["status"], vote["id"], vote["support"], vote["votes"],"## I do not support this proposal: " + vote["reason"]]])], ignore_index=True)

# rename the columns
reasons_df.columns = ['proposal_id','quorumVotes','forVotes','againstVotes', 'abstainVotes','description','status','voter id','voter support','votes','reason']

In [28]:
# print the shape of the new dataframe
print(reasons_df.shape)
# print all the unique proposal ids sorted low to high
print(reasons_df["proposal_id"].sort_values().unique().tolist())
# print how many unique proposal ids there are
print(len(reasons_df["proposal_id"].sort_values().unique().tolist()))

(271, 11)
['44', '47', '49', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '67', '68', '69', '70', '71', '72', '73', '74', '75', '77', '78', '81', '82', '83', '84', '85', '86', '87', '89', '90', '91', '92', '93', '94', '95', '97', '98', '99']
44


In [29]:
reasons_df.head()

,proposal_id,quorumVotes,forVotes,againstVotes,abstainVotes,description,status,voter id,voter support,votes,reason
0,44,24,78,1,1,# Proposal: # Nouns Acquisition Committee ## ...,EXECUTED,0xcb6fbf9b4e822bed1a8365c0c32e66bd7a779bae-44,True,0,## I support this proposal: I am in favor
1,49,26,33,9,28,# Proposal: # Additional Funding for Nouns Acq...,EXECUTED,0x5306c064f74b2c45d3f1afae90cf0d74f7523fe4-49,False,0,## I do not support this proposal: stated purp...
2,53,26,75,0,1,# Proposal: # Contribute 6.9E to funding Calla...,EXECUTED,0xcc2688350d29623e2a0844cc8885f9050f0f6ed5-53,True,7,## I support this proposal: we liked it
3,57,27,48,8,16,# Proposal: # Bring luxury Noun sunglasses to ...,EXECUTED,0x008c84421da5527f462886cec43d2717b686a7e4-57,False,5,## I do not support this proposal: For the Pro...
4,57,27,48,8,16,# Proposal: # Bring luxury Noun sunglasses to ...,EXECUTED,0x2573c60a6d127755aa2dc85e342f7da2378a0cc5-57,True,26,## I support this proposal: ⌐◨-◨


In [31]:
# print length of the longest "reason"
print(max([len(x) for x in reasons_df["reason"]]))

8317


In [15]:
# print the full "reason" of any row with a duplicate reason
reasons_df.loc[reasons_df.duplicated(subset="reason", keep=False)]

,proposal_id,quorumVotes,forVotes,againstVotes,abstainVotes,description,status,voter id,voter support,votes,reason
4,57,27,48,8,16,# Proposal: # Bring luxury Noun sunglasses to ...,EXECUTED,0x2573c60a6d127755aa2dc85e342f7da2378a0cc5-57,True,26,## I support this proposal: ⌐◨-◨
36,62,27,71,0,0,# Proposal: # Prop House Continuation **tl;dr...,EXECUTED,0x2573c60a6d127755aa2dc85e342f7da2378a0cc5-62,True,25,## I support this proposal: ⌐◨-◨
184,92,33,126,1,0,# Proposal: # David Horvath / Nouns Studio1 ex...,EXECUTED,0xcc2688350d29623e2a0844cc8885f9050f0f6ed5-92,True,13,## I support this proposal: ⌐◨-◨
229,98,34,42,0,6,# Proposal: # NOUNS IN RIO - Nouns to be a par...,QUEUED,0xcc2688350d29623e2a0844cc8885f9050f0f6ed5-98,True,12,## I support this proposal: ⌐◨-◨
236,99,35,77,2,0,"# Proposal: # Sailing PR campaign, Korea Block...",QUEUED,0xa68fdb0c7ecdc404637c5c06bab82a7bf00609f3-99,True,1,## I support this proposal: ⌐◨-◨
245,56,27,39,1,2,# Proposal: # FUN Noggles Stage 2 ### SharkDA...,EXECUTED,0x8323f1c687f7e2296ec71ee3549a7430ea7ec730-56,True,1,## I support this proposal: ⌐◨-◨


In [16]:
# rename 'description' to 'prompt' and 'reason' to 'completion'
reasons_df.rename(columns={"description": "prompt", "reason": "completion"}, inplace=True)

In [17]:
# save the dataframe to a csv file
reasons_df.to_csv("nouns_vote_reasons.csv", index=False)

In [167]:
!openai tools fine_tunes.prepare_data -f .\nouns_vote_reasons.csv

'openai' is not recognized as an internal or external command,
operable program or batch file.


In [48]:
prompt = """
# Proposal: # The Noun Square: A Daily Noun O' Clock Celebration
Summary
The Noun Square is a daily Twitter Spaces ritual celebrating Noun O' Clock (NOC) in a fun and engaging format.
Spearheaded by Toady Hawk, Ser Noadz and myself (142). Administered by a thriving and growing community (join our discord!).
We've completed a 14 day pilot of daily spaces and contests every day, with excellent results.
Pilot Metrics
~110 daily attendees, 60+ daily FOMO players
500+ twitter followers (up from 0)
250K tweet impressions (!!)
200+ pieces of nounish content produced in 9 daily contests
400+ POAPs claimed for attending the spaces
11 special nounish guests
Short term goals
Grow attendance & engagement
Iterate on show format and deepen our "show/market fit"
Grow our community of hosts and contributors
Longer term goals
Make NOC a global Internet shared moment
Grow reach to thousands of concurrent participants, and later much much more
Our Ask
We are asking for 55 ETH total to continue this initiative for a 1 month period.
Doing this daily is a lot of work (about 10 man-hour a day on average. More detailed breakdown below.)
10 ETH will be earmarked for contests themed around the Noun of the day.
45 ETH will go to community contributor rewards (biweekly retro rewards administered by CoordinApe rounds, modeled after Nouncil.)

Who We Are
We are a community of Nounish builders setting out to create a Daily Celebration of Noun O’ Clock called The Noun Square, administered by a Nouncil-like body and with the goal of proliferating Nouns by turning Noun O’ Clock into a “Shared Moment” for the entire Internet. We have successfully completed two week pilot using a combination of Small Grants and out-of-pocket funds for prizes (data from which which has informed this proposal)

Our vision
For anyone who grew up in the 70s, 80s or 90s, “The Price is Right” holds a special place in our collective psyche. For over 50 years, its daily blend of ritual and play has entertained hundreds of thousands of people the world over. We believe Noun O’ Clock can recreate this kind of shared experience for web3.

Broad Goals:
Turn NOC into a daily shared global moment - develop a grass-root, organic, daily show that introduces more people to nouns & its thriving community & sub-communities. As well as creates a space in which our growing community can meet daily and share a live moment together.
Model our community on Nouncil - a collective of builders all over the globe with varied skillsets, collaborating to organize and administer a daily Twitter Spaces. We think that a decentralized community model can be the nounish way to scale a Noun O' Clock show!
Be consistent (”one NOC, every day, forever”) - Even in the deep NOC winter, we believe consistency is key to cementing NOC as a daily celebration for the Internet. We are planning ahead for this with hosts from all over the world.

The (Current) Format
Begin 15 min before NOC every day.
Contest review & announcements.
Casual guest interview while watching/discussing end of daily Noun auction.
Play FOMO Live together (we invite as many attendees as possible to come on stage and keep mics open for an interactive experience.)
Give live Secret Word for those who wish to collect POAP for attendance.
*On weekends, we run a LITE version of The Noun Square featring only FOMO live, hosted by a different nounish community every Saturday (e.g. Lil Nouns and GOOP DAO the past two weeks, future guest hosts Nouns DAO Japan, Korean Noun Fans, NounPunks, FloriNouns, etc.)

The Team: 3 C's
Core Team (Responsible for Execution):
Toady Hawk
Noun 142
Ser Noadz

Community (Talent Funnel for Council)
Growing Discord Community

Growing the community
We want to grow the community and build in the open. We have created a Discord server open to anyone who wants to contribute. We have also created a public Google Sheet that tracks our progress and metrics and future plans for anyone to follow along.
Hosting a live Spaces every day is an audacious goal (albeit one we are excited about), and will require participation and help from Nounsfolk in all timezones, especially as NOC drifts into the North American Night.

Compensation Model
Our ask for the next 30 days is 55 ETH:
Contest rewards (0.5 eth/day * 20 weekdays = 10 ETH)
We will experiment with these numbers and the cadence of them as we move forward, but so far this amount and cadence has produced excellent participation.
Team reward + operations (see budget below): 45 ETH
Because we intend to build this initiative as a community effort, we will be modeling our team reward, at least for the first period, on Nouncil’s proven model of a Coordinape round, where all active participants will allocate tokens based on contributions.** We think this will ultimately be a fairer and more frictionless way of properly rewarding the team than attempting to itemize every hour of effort, etc.
Based on our pilot, we estimated how many man-hours it takes to run a show like this every day. We think these are reasonable overall estimates.

Two-week Pilot Results
Twitter analytics
Massive reach within 14 days!
 
Gamifying NOC with POAPs
We are using POAPs to gamify NOC and create both engagement and loyalty. We feel that our use of Daily Attendance POAPs is the purest use case for the tech (literally “Proof of Attendance”) and we want to use them to incentivize people to attend Noun O’ Clock every single day if they can.
So far, it’s working! Ser Noadz has been creating amazing daily animated POAPs based on the new Noun of the Day and would continue to do so for the first month (after which we would explore other “Series” by other Nounish artists.)
 
We also want to experiment with doing bonuses like physical Merch drops for those with provable streaks of attendance, and raffles that are easier to win the more times you have attended.
Example Possible “The Noun Square” Merch Drop for Attendance Streaks:
 
Playing FOMO
The peak moment of The Noun Square is, of course, playing FOMO live and minting a new Noun together. When FOMO starts during the Twitter Space, we ask all speakers to unmute and speak their mind, to create as much chaos, interaction and fun as possible. The feedback so far has been that people have been loving it!
We have noticed anecdotally that FOMO seems to take longer on average with more players, until we hit that beautiful noun that reaches some consensus. We think the Nouns minted over the past week or so are especially legendary, and believe it’s related to our efforts to gamify FOMO and bring in new players.
The impressive roster of Nouns minted by FOMO since we started:
 

Potential challenges
1.	July gets into very late night EST and we may have fewer participants. However, we believe that consistency is key, as well as the opportunity to get more people from non-north america timezones to hear about nouns.
2.	Over-saturation is a risk of course, as with anything that happens daily. But as with The Price is Right, we hope to create a shared ritual with the community we are building, such that the daily show is something they look forward to, rather than a chore they get bored of. We are building variety into the structure of the show, with a mixture of weekly segments and novel guests and contests to keep things interesting for many, many months (and hopefully longer!)
In Conclusion:
Over the past two weeks, we have kicked off a tradition of making Noun O’ Clock a daily celebration, featuring live Twitter Spaces, interactive contests and special guests. So far, people love it. (See feedback section below)
We believe that, if given funding to execute our vision detailed here, we can bring new attention to Nouns and the daily auction every day, while also supporting extension and sub-DAO communities, and engaging active Nounish builders.
Let’s work together to make Noun O’ Clock into a shared moment for the Internet!

\n\n###\n\n

## I
"""

In [57]:
import openai
max_tokens = 150

openai.Completion.create(
    model="ada:ft-personal-2022-06-29-00-28-59",
    prompt=prompt, max_tokens = max_tokens)


<OpenAIObject text_completion id=cmpl-5OG8D3EZFkRXPERGvfrK2xJUmEXc1 at 0x1fabf5bf880> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "### I support this proposal: Endorsed by Team Toady and Ser Noadz ENDORSED\nThis is a great opportunity to experiment with such a small grant and experiment in hiring, monetizing and running the pilot. This platform is still growing, so adding more staff rapidly, hiring more professional contributors before the pilot expires, and ensuring we have the community and quantity needed, alongside DDO client, is important to make this work. The dollar amount is not unreasonable and it covers both an hourly rate and operations. Endorsers Kalen, Ser Noadz & Toady. Computers Iphone and Google Chrome. \nCTO Fees: 55 ETH\n**System Palo Alto Research Center for calculating computational costs \n!!"
    }
  ],
  "created": 1656463565,
  "id": "cmpl-5OG8D3EZFkRXPERGvfrK2xJUmEXc1",
  "model": "ada:ft